In [1]:
from rosetta import *
init()

core.init: Rosetta version  from 
core.init: command: PyRosetta -ex1 -ex2aro -database /Users/XT/ResearchApps/PyRosetta_1230/database
core.init: 'RNG device' seed mode, using '/dev/urandom', seed=818306491 seed_offset=0 real_seed=818306491
core.init.random: RandomGenerator:init: Normal mode, seed=818306491 RG_type=mt19937


In [2]:
seq='NFYGELVDLGVKEKLIEKAGAWYSYKGEKIGQGKANATAWLKDNPETAKEIEKKVRELLL'

In [27]:
initial_pose=pose_from_sequence(seq)
pose=Pose()
pose.assign(initial_pose)

In [29]:
pmm=PyMolMover()
pmm.apply(pose)

In [5]:
kT=1
mm = MoveMap()
mm.set_bb(True)

fragset9 = ConstantLengthFragSet(9)
fragset9.read_fragment_file("aat000_09_05.200_v1_3")
mover_9mer = ClassicFragmentMover(fragset9, mm)
fragset3 = ConstantLengthFragSet(3)
fragset3.read_fragment_file("aat000_03_05.200_v1_3")
mover_3mer = ClassicFragmentMover(fragset3, mm)



core.fragments.ConstantLengthFragSet: finished reading top 200 9mer fragments from file aat000_09_05.200_v1_3
core.fragments.ConstantLengthFragSet: finished reading top 200 3mer fragments from file aat000_03_05.200_v1_3


In [30]:
# protocol
switch = SwitchResidueTypeSetMover("centroid")
switch.apply(pose)

centroid_scorefxn=get_cen_scorefxn()
mc_low=MonteCarlo(pose,centroid_scorefxn,kT)
pmm.apply(pose)

In [31]:
for iteration in range(1000):
    mover_9mer.apply(pose)
    mc_low.boltzmann(pose)
for iteration in range(2000):
    mover_3mer.apply(pose) 
    mc_low.boltzmann(pose)
    pmm.apply(pose)


True

In [ ]:
pose.dump_pdb('Low_Res_decoy.pdb')

In [32]:
switch = SwitchResidueTypeSetMover("fa_standard")
switch.apply(pose)

scorefxn=get_fa_scorefxn()
mc=MonteCarlo(pose,scorefxn,kT)
smallmover=SmallMover(mm,kT,5)
shearmover=ShearMover(mm,kT,5)
minmover=MinMover()
minmover.movemap(mm)   
minmover.score_function(scorefxn)
minmover.min_type('dfpmin')

core.scoring.ScoreFunctionFactory: SCOREFUNCTION: talaris2013


In [34]:
for minmove in range(10):
    for torsion_mover in range(100):
        smallmover.apply(pose)
        shearmover.apply(pose)
        mc.boltzmann(pose)
    minmover.apply(pose)
    mc.boltzmann(pose)
    pmm.apply(pose)

    

In [35]:
pose.dump_pdb('High_Res_decoy.pdb')

True